<a href="https://colab.research.google.com/github/Navyamohan909/Elevate_Labs/blob/main/News%20Article%20Classification%20(Fake/Real).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "2xUkG9pLbr02Vhqgmua7aNDcvjo_3b5FXikvrYLb2zG3UVfpW"


In [ ]:
# Step 1: Install required libraries
!pip install flask pyngrok pandas nltk scikit-learn joblib -q

# Step 2: Imports
import pandas as pd
import string
import nltk
import joblib
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import os

# Step 3: Download stopwords
nltk.download('stopwords')

# Step 4: Define text cleaning function
def clean_text_for_prediction(text_input):
    text_input = str(text_input).lower()
    text_input = ''.join([c for c in text_input if c not in string.punctuation])
    tokens = text_input.split()
    tokens = [w for w in tokens if w not in stopwords.words('english')]
    return ' '.join(tokens)

# Step 5: Load model and vectorizer
def load_model_and_vectorizer():
    global loaded_model, loaded_vectorizer
    model_path = 'news_model.pkl'
    vectorizer_path = 'vectorizer.pkl'

    if not os.path.exists(model_path) or not os.path.exists(vectorizer_path):
        print("Please upload 'news_model.pkl' and 'vectorizer.pkl'")
        try:
            from google.colab import files
            uploaded = files.upload()
        except ImportError:
            print("Not in Colab. Upload manually.")
            return False

    try:
        loaded_model = joblib.load(model_path)
        loaded_vectorizer = joblib.load(vectorizer_path)
        print("✅ Model and vectorizer loaded.")
        return True
    except Exception as e:
        print(f"❌ Error loading model/vectorizer: {e}")
        return False

# Step 6: Create Flask app
def create_flask_app():
    global flask_app
    flask_app = Flask(__name__)

    html_template = """
    <!DOCTYPE html><html><head><title>News Checker</title></head>
    <body style="font-family:sans-serif;padding:20px">
        <h2>📰 News Authenticity Checker</h2>
        <form method="post">
            <textarea name="news_text" rows="10" cols="80" placeholder="Paste news text here...">{{ S_TEXT if S_TEXT else '' }}</textarea><br><br>
            <input type="submit" value="Check News">
        </form>
        {% if PRED_RESULT %}
            <h3 style="color:{{ 'green' if PRED_RESULT=='REAL' else 'red' }}">Prediction: {{ PRED_RESULT }}</h3>
        {% endif %}
    </body></html>
    """

    @flask_app.route("/", methods=["GET", "POST"])
    def home():
        prediction_result = None
        user_input = None
        if request.method == "POST":
            news_text = request.form.get("news_text", "").strip()
            user_input = news_text
            if news_text:
                try:
                    cleaned_text = clean_text_for_prediction(news_text)
                    vectorized_input = loaded_vectorizer.transform([cleaned_text])
                    pred = loaded_model.predict(vectorized_input)[0]
                    prediction_result = "REAL" if pred == 1 else "FAKE"
                except Exception as e:
                    prediction_result = f"Error: {e}"
        return render_template_string(html_template, PRED_RESULT=prediction_result, S_TEXT=user_input)

# Step 7: Main Execution
if __name__ == "__main__":
    print("🔁 Starting Execution...")
    if load_model_and_vectorizer():
        create_flask_app()

        # 🔐 Set ngrok token
        NGROK_AUTH_TOKEN = "2xUkG9pLbr02Vhqgmua7aNDcvjo_3b5FXikvrYLb2zG3UVfpW"
        if NGROK_AUTH_TOKEN and "2xUkG9pLbr02Vhqgmua7aNDcvjo_3b5FXikvrYLb2zG3UVfpW" not in NGROK_AUTH_TOKEN:
            try:
                ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            except Exception as e:
                print(f"Warning: Could not set ngrok auth token: {e}")

        try:
            ngrok.kill()  # Clean any old tunnels
        except:
            pass

        try:
            public_url = ngrok.connect(5000)
            print(f"🌐 App is running at: {public_url}")
            print("⚠ Keep this cell running to keep the app online.")
            flask_app.run(port=5000)
        except Exception as e:
            print(f"❌ Could not launch app: {e}")
    else:
        print("❌ Could not load model or vectorizer. Please check file names.")


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

🔁 Starting Execution...
✅ Model and vectorizer loaded.


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


🌐 App is running at: NgrokTunnel: "https://aa26-34-80-59-65.ngrok-free.app" -> "http://localhost:5000"
⚠ Keep this cell running to keep the app online.
 * Serving Flask app '__main__'
 * Debug mode: off


Streaming output truncated to the last 5000 lines.
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed